# Import modules

In [2]:
import numpy as np
import pingouin as pg
import pandas as pd
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import statsmodels.api as sm
import statsmodels.formula.api as smf


/usr/local/lib/python3.8/dist-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Functions (helpers)

In [3]:
def read_csv(name, info=True):
    df = pd.read_csv(f'../data/stats/{name}.csv')
    # pd.set_option('display.max_columns', None)
    # pd.set_option('max_columns', None)
    if info:
        print_section('DATASET INFO')
        print('N rows=', df.size)
        print('N cols=', len(df.columns))
        print(df.head())
    return df

def print_section(title):
    print('\n')
    print( '*' + '-' * 135 + '*')
    print('* ' + title)
    print( '*' + '-' * 135 + '*')

def print_subsection(title):
    print('\n')
    print('***** ' + title + ' *****')
    print( '*' + '-' * 135 + '*')

# Fig 2

 * load data

In [4]:
filename = 'Fig2A'
df = read_csv(filename)
pd.options.display.float_format = '{:,.5f}'.format




*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 7672
N cols= 7
   subject  exp_num  complete  block  less_cues     score  cond
0        1        1         0      0          0  0.166667     4
1        2        1         0      0          0  0.866667     4
2        3        1         0      0          0  0.466667     4
3        4        1         0      0          0  0.400000     4
4        5        1         0      0          0  0.733333     4


* Test for performance above chance in experiments 1:4 (Fig. 2A)

In [5]:
table = []
for exp_num in (1, 2, 3, 4):
    for cond in (1, 2, 3, 4):
        if exp_num == 4 and cond not in (1, 4):
            continue
        row = pg.ttest(df['score'][df.exp_num==exp_num][df.cond==cond], 0.5, confidence=0.975)
        row.insert(loc=0, column='exp_num', value=exp_num)
        row.insert(loc=1, column='cond', value=cond)
        table.append(row)

df2 = pd.concat(table)
df2

,exp_num,cond,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,1,1,10.98380,75,two-sided,0.00000,"[0.7014120273818355, 0.807359902442726]",1.25993,2.102e+14,1.00000
T-test,1,2,6.99011,75,two-sided,0.00000,"[0.6192121697203697, 0.735173795191911]",0.80182,1.093e+07,1.00000
T-test,1,3,7.00086,75,two-sided,0.00000,"[0.6145764055032893, 0.7257744716896933]",0.80305,1.143e+07,1.00000
T-test,1,4,1.50661,75,two-sided,0.13611,"[0.47818208358845515, 0.6060284427273339]",0.17282,0.372,0.31852
T-test,2,1,19.43878,70,two-sided,0.00000,"[0.7981954029463831, 0.8778609350817859]",2.30696,4.82e+26,1.00000
T-test,2,2,11.65526,70,two-sided,0.00000,"[0.7074681199974906, 0.8089638048851382]",1.38323,1.206e+15,1.00000
T-test,2,3,7.72376,70,two-sided,0.00000,"[0.6251642616792291, 0.7307042829217097]",0.91664,1.62e+08,1.00000
T-test,2,4,3.62985,70,two-sided,0.00054,"[0.5322177124523573, 0.6424301748715864]",0.43078,44.471,0.94735
T-test,3,1,27.05025,82,two-sided,0.00000,"[0.8724871125444486, 0.9411675059294469]",2.96915,1.549e+39,1.00000
T-test,3,2,19.91188,82,two-sided,0.00000,"[0.8189319587756545, 0.9015499689351889]",2.18561,7.365e+29,1.00000


* Difficult vs Easy choice problems pairwise ttests

In [6]:

pg.pairwise_tests(data=df, dv='score',  subject='subject', between='exp_num', within='cond', padjust='bonf', effsize='cohen', within_first=False)


pg.ttest(df[(df.exp_num==1) &  (df.cond==1)].score, df[ (df.exp_num==1) &  (df.cond==4) ].score, paired=True, confidence=0.975 )
pg.ttest(df[(df.exp_num==2) &  (df.cond==1)].score, df[ (df.exp_num==2) &  (df.cond==4) ].score, paired=True, confidence=0.975 )

,Contrast,exp_num,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,exp_num,-,1.00000,2.00000,False,True,-2.69270,143.24161,two-sided,0.00793,0.02380,bonf,4.704,-0.44173
1,exp_num,-,1.00000,3.00000,False,True,-7.65359,156.48001,two-sided,0.00000,0.00000,bonf,3.523e+09,-1.21345
2,exp_num,-,2.00000,3.00000,False,True,-5.48549,151.56326,two-sided,0.00000,0.00000,bonf,7.315e+04,-0.87240
3,cond,-,1.00000,2.00000,True,True,5.74184,229.00000,two-sided,0.00000,0.00000,bonf,3.014e+05,0.35073
4,cond,-,1.00000,3.00000,True,True,9.04336,229.00000,two-sided,0.00000,0.00000,bonf,7.78e+13,0.61669
5,cond,-,1.00000,4.00000,True,True,11.60547,229.00000,two-sided,0.00000,0.00000,bonf,4.361e+21,0.96845
6,cond,-,2.00000,3.00000,True,True,3.40183,229.00000,two-sided,0.00079,0.00474,bonf,19.493,0.25176
7,cond,-,2.00000,4.00000,True,True,7.42745,229.00000,two-sided,0.00000,0.00000,bonf,3.032e+09,0.61596
8,cond,-,3.00000,4.00000,True,True,4.42752,229.00000,two-sided,0.00001,0.00009,bonf,801.388,0.38100
9,exp_num * cond,1,1,2,True,True,2.82718,75.00000,two-sided,0.00602,0.10832,bonf,4.973,0.36470


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,5.81453,75,two-sided,0.00000,"[0.12877474063225536, 0.2957866628765168]",0.94876,9.578e+04,1.00000


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,8.81057,70,two-sided,0.00000,"[0.18552420897607175, 0.31588424172815366]",1.41751,1.368e+10,1.00000


* Test for performance within Exp. 1:4 across conditions (pairs of symbols) (Fig. 2A)

In [7]:
# test performance within exp and across cond with regression
print_section('regress performance across conditions within experiments')

x = df[['cond', 'exp_num']]
y = df[['exp_num', 'score']]

table = []
for exp_num in (1, 2, 3, 4):
    
    row = pg.linear_regression(
        x[x.exp_num==exp_num].cond, y[y.exp_num==exp_num].score, add_intercept=True, as_dataframe=False)

    row['dof_model'] = [row['df_model'], ] * len(row['coef'])
    row['dof_resid'] = [row['df_resid'], ] * len(row['coef'])

    del row['residuals']; del row['df_model']
    del row['df_resid']; del row['X']; del row['y']; del row['pred']
    row = pd.DataFrame(row)
    
    row.insert(loc=0, column='exp_num', value=exp_num)
    table.append(
        row
    )

df3 = pd.concat(table)
df3

# ANOVA control
print_section('Anova performance across conditions within experiments')
table = []
for exp_num in (1, 2, 3, 4):
    
    row = pg.rm_anova(data=df, dv='score', within='cond', detailed=True, subject='subject', correction=True)

    row = pd.DataFrame(row)
    
    row.insert(loc=0, column='exp_num', value=exp_num)
    table.append(
        row
    )

df4 = pd.concat(table)
df4






*---------------------------------------------------------------------------------------------------------------------------------------*
* regress performance across conditions within experiments
*---------------------------------------------------------------------------------------------------------------------------------------*


,exp_num,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%],dof_model,dof_resid
0,1,Intercept,0.82193,0.03102,26.49265,0.00000,0.09662,0.09363,0.76088,0.88298,1,302
1,1,cond,-0.06439,0.01133,-5.68346,0.00000,0.09662,0.09363,-0.08668,-0.04209,1,302
0,2,Intercept,0.92347,0.02662,34.69104,0.00000,0.20638,0.20357,0.87108,0.97587,1,282
1,2,cond,-0.08324,0.00972,-8.56352,0.00000,0.20638,0.20357,-0.10237,-0.06411,1,282
0,3,Intercept,0.96486,0.02540,37.98767,0.00000,0.09879,0.09606,0.91489,1.01482,1,330
1,3,cond,-0.05578,0.00927,-6.01467,0.00000,0.09879,0.09606,-0.07403,-0.03754,1,330
0,4,Intercept,0.96521,0.03172,30.42501,0.00000,0.18856,0.18390,0.90260,1.02783,1,174
1,4,cond,-0.06919,0.01088,-6.35875,0.00000,0.18856,0.18390,-0.09067,-0.04772,1,174




*---------------------------------------------------------------------------------------------------------------------------------------*
* Anova performance across conditions within experiments
*---------------------------------------------------------------------------------------------------------------------------------------*


,exp_num,Source,SS,DF,MS,F,p-unc,p-GG-corr,ng2,eps,sphericity,W-spher,p-spher
0,1,cond,5.22892,3,1.74297,56.85023,0.00000,0.00000,0.11422,0.80801,False,0.67885,0.00000
1,1,Error,21.06275,687,0.03066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,cond,5.22892,3,1.74297,56.85023,0.00000,0.00000,0.11422,0.80801,False,0.67885,0.00000
1,2,Error,21.06275,687,0.03066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3,cond,5.22892,3,1.74297,56.85023,0.00000,0.00000,0.11422,0.80801,False,0.67885,0.00000
1,3,Error,21.06275,687,0.03066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4,cond,5.22892,3,1.74297,56.85023,0.00000,0.00000,0.11422,0.80801,False,0.67885,0.00000
1,4,Error,21.06275,687,0.03066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Test performance increase across Exp 1:4 (Fig. 2A)

In [8]:
# test performance within exp and across cond with mixed model
print_section('regress performance across experiments')
df1 = df[df.exp_num==1]

df1['cond']  = df1['cond'].map({1: 4, 2: 3, 3:2, 4:1})
smf.mixedlm('score ~ C(cond)', groups='subject', data=df1).fit().summary()

df_ind = df.groupby('subject').mean()
smf.ols('score ~ C(exp_num)', data=df_ind).fit().summary()
# res.summary()

23.41**2



*---------------------------------------------------------------------------------------------------------------------------------------*
* regress performance across experiments
*---------------------------------------------------------------------------------------------------------------------------------------*


/tmp/ipykernel_13564/3010239230.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cond']  = df1['cond'].map({1: 4, 2: 3, 3:2, 4:1})
/usr/local/lib/python3.8/dist-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
        Mixed Linear Model Regression Results
======================================================
Model:             MixedLM Dependent Variable: score  
No. Observations:  304     Method:             REML   
No. Groups:        76      Scale:              0.0409 
Min. group size:   4       Log-Likelihood:     24.3443
Max. group size:   4       Converged:          Yes    
Mean group size:   4.0                                
------------------------------------------------------
             Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------
Intercept    0.542    0.025 21.467 0.000  0.493  0.592
C(cond)[T.2] 0.128    0.033  3.902 0.000  0.064  0.192
C(cond)[T.3] 0.135    0.033  4.116 0.000  0.071  0.199
C(cond)[T.4] 0.212    0.033  6.468 0.000  0.148  0.277
subject Var  0.008    0.017                           
======================================================

"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     21.02
Date:                Thu, 13 Oct 2022   Prob (F-statistic):           1.96e-12
Time:                        11:27:14   Log-Likelihood:                 166.18
No. Observations:                 318   AIC:                            -324.4
Df Residuals:                     314   BIC:                            -309.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.6610      0.017     39.905      0.000       0.628       0.694
C(exp_num)[T.2.0]     0.0544      0.024      2.283      0.023       0.008       0.101
C(exp_num)[T.3.0]     0.1644      0.023      7.173      0.000       0.119       0.210
C(exp_num)[T.4.0]     0.1313      0.023      5.805      0.000       0.087       0.176
==============================================================================
Omnibus:                       55.410   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              105.354
Skew:                          -0.936   Prob(JB):                     1.33e-23
Kurtosis:                       5.108   Cond. No.                         4.90
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

548.0281

* Mean accuracy for exp 1-4

In [9]:
# print mean and variance of each exp
table = [] 
for exp_num in (1, 2, 3, 4):
    table.append(
        {
            'Exp. ': exp_num,
            'mean': df_ind[df_ind.exp_num==exp_num].score.mean(),
            'sem': df_ind[df_ind.exp_num==exp_num].score.sem(),
        }
    )
pd.DataFrame(table)
            

,Exp.,mean,sem
0,1,0.66096,0.01529
1,2,0.71538,0.01321
2,3,0.82540,0.01509
3,4,0.79223,0.01911



* Fig 2D, LE-ES-SP slope increase

In [10]:
filename = 'Fig2D'
df = read_csv(filename)
pd.options.display.float_format = '{:,.5f}'.format
# test performance within exp and across cond with regression
print_section('Regress slopes across experiments')
#df['modality'] = df['modality'].map({'LE': 1, 'ES': 2, 'SP':3})
df 

# iterate through each modality and set it as "reference modality" to get "absolute slopes" for each
for modality in ('LE', 'ES', 'SP'):
    print_subsection('Modality used as reference= ' + str(modality))
    #smf.ols(f'slope ~ C(exp_num) * C(modality, Treatment("{modality}"))', data=df).fit().summary()

    smf.mixedlm(f'slope ~ C(exp_num) * C(modality, Treatment("{modality}"))', groups='subject', data=df).fit().summary()




*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 3816
N cols= 4
   subject  exp_num   slope modality
0        1        1 0.42728       LE
1        2        1 0.90420       LE
2        3        1 0.43349       LE
3        4        1 0.65978       LE
4        5        1 0.84207       LE


*---------------------------------------------------------------------------------------------------------------------------------------*
* Regress slopes across experiments
*---------------------------------------------------------------------------------------------------------------------------------------*


,subject,exp_num,slope,modality
0,1,1,0.42728,LE
1,2,1,0.90420,LE
2,3,1,0.43349,LE
3,4,1,0.65978,LE
4,5,1,0.84207,LE
...,...,...,...,...
949,314,4,0.33824,SP
950,315,4,0.11765,SP
951,316,4,1.11765,SP
952,317,4,1.13235,SP




***** Modality used as reference= LE *****
*---------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Mixed Linear Model Regression Results
=============================================================================================
Model:                          MixedLM             Dependent Variable:             slope    
No. Observations:               954                 Method:                         REML     
No. Groups:                     318                 Scale:                          0.0855   
Min. group size:                3                   Log-Likelihood:                 -344.4469
Max. group size:                3                   Converged:                      Yes      
Mean group size:                3.0                                                          
---------------------------------------------------------------------------------------------
                                                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------
Intercept                                           0.590    0.041 14.466 0.000  0.510  0.670
C(exp_num)[T.2]                                     0.156    0.059  2.657 0.008  0.041  0.271
C(exp_num)[T.3]                                     0.285    0.056  5.056 0.000  0.175  0.396
C(exp_num)[T.4]                                     0.309    0.056  5.558 0.000  0.200  0.419
C(modality, Treatment("LE"))[T.ES]                 -0.366    0.047 -7.713 0.000 -0.459 -0.273
C(modality, Treatment("LE"))[T.SP]                 -0.270    0.047 -5.699 0.000 -0.363 -0.177
C(exp_num)[T.2]:C(modality, Treatment("LE"))[T.ES] -0.257    0.068 -3.769 0.000 -0.391 -0.124
C(exp_num)[T.3]:C(modality, Treatment("LE"))[T.ES] -0.262    0.066 -3.989 0.000 -0.391 -0.133
C(exp_num)[T.4]:C(modality, Treatment("LE"))[T.ES] -0.197    0.065 -3.036 0.002 -0.324 -0.070
C(exp_num)[T.2]:C(modality, Treatment("LE"))[T.SP] -0.250    0.068 -3.658 0.000 -0.384 -0.116
C(exp_num)[T.3]:C(modality, Treatment("LE"))[T.SP] -0.194    0.066 -2.948 0.003 -0.322 -0.065
C(exp_num)[T.4]:C(modality, Treatment("LE"))[T.SP] -0.128    0.065 -1.978 0.048 -0.255 -0.001
subject Var                                         0.041    0.023                           
=============================================================================================

"""



***** Modality used as reference= ES *****
*---------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Mixed Linear Model Regression Results
=============================================================================================
Model:                          MixedLM             Dependent Variable:             slope    
No. Observations:               954                 Method:                         REML     
No. Groups:                     318                 Scale:                          0.0855   
Min. group size:                3                   Log-Likelihood:                 -344.4469
Max. group size:                3                   Converged:                      Yes      
Mean group size:                3.0                                                          
---------------------------------------------------------------------------------------------
                                                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------
Intercept                                           0.224    0.041  5.493 0.000  0.144  0.304
C(exp_num)[T.2]                                    -0.101    0.059 -1.727 0.084 -0.216  0.014
C(exp_num)[T.3]                                     0.023    0.056  0.416 0.678 -0.087  0.134
C(exp_num)[T.4]                                     0.113    0.056  2.026 0.043  0.004  0.222
C(modality, Treatment("ES"))[T.LE]                  0.366    0.047  7.713 0.000  0.273  0.459
C(modality, Treatment("ES"))[T.SP]                  0.096    0.047  2.014 0.044  0.003  0.189
C(exp_num)[T.2]:C(modality, Treatment("ES"))[T.LE]  0.257    0.068  3.769 0.000  0.124  0.391
C(exp_num)[T.3]:C(modality, Treatment("ES"))[T.LE]  0.262    0.066  3.989 0.000  0.133  0.391
C(exp_num)[T.4]:C(modality, Treatment("ES"))[T.LE]  0.197    0.065  3.036 0.002  0.070  0.324
C(exp_num)[T.2]:C(modality, Treatment("ES"))[T.SP]  0.008    0.068  0.111 0.912 -0.126  0.141
C(exp_num)[T.3]:C(modality, Treatment("ES"))[T.SP]  0.068    0.066  1.041 0.298 -0.060  0.197
C(exp_num)[T.4]:C(modality, Treatment("ES"))[T.SP]  0.069    0.065  1.059 0.290 -0.058  0.196
subject Var                                         0.041    0.023                           
=============================================================================================

"""



***** Modality used as reference= SP *****
*---------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Mixed Linear Model Regression Results
=============================================================================================
Model:                          MixedLM             Dependent Variable:             slope    
No. Observations:               954                 Method:                         REML     
No. Groups:                     318                 Scale:                          0.0855   
Min. group size:                3                   Log-Likelihood:                 -344.4469
Max. group size:                3                   Converged:                      Yes      
Mean group size:                3.0                                                          
---------------------------------------------------------------------------------------------
                                                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------
Intercept                                           0.320    0.041  7.837 0.000  0.240  0.400
C(exp_num)[T.2]                                    -0.094    0.059 -1.598 0.110 -0.209  0.021
C(exp_num)[T.3]                                     0.092    0.056  1.627 0.104 -0.019  0.202
C(exp_num)[T.4]                                     0.181    0.056  3.257 0.001  0.072  0.291
C(modality, Treatment("SP"))[T.ES]                 -0.096    0.047 -2.014 0.044 -0.189 -0.003
C(modality, Treatment("SP"))[T.LE]                  0.270    0.047  5.699 0.000  0.177  0.363
C(exp_num)[T.2]:C(modality, Treatment("SP"))[T.ES] -0.008    0.068 -0.111 0.912 -0.141  0.126
C(exp_num)[T.3]:C(modality, Treatment("SP"))[T.ES] -0.068    0.066 -1.041 0.298 -0.197  0.060
C(exp_num)[T.4]:C(modality, Treatment("SP"))[T.ES] -0.069    0.065 -1.059 0.290 -0.196  0.058
C(exp_num)[T.2]:C(modality, Treatment("SP"))[T.LE]  0.250    0.068  3.658 0.000  0.116  0.384
C(exp_num)[T.3]:C(modality, Treatment("SP"))[T.LE]  0.194    0.066  2.948 0.003  0.065  0.322
C(exp_num)[T.4]:C(modality, Treatment("SP"))[T.LE]  0.128    0.065  1.978 0.048  0.001  0.255
subject Var                                         0.041    0.023                           
=============================================================================================

"""

* Fig 2D, LE-ES-SP slope comparisons

In [11]:
filename = 'Fig2D'
df = read_csv(filename)
pd.options.display.float_format = '{:,.5f}'.format
pg.pairwise_ttests(dv='slope', subject='subject', between='exp_num', within='modality', effsize='cohen', data=df, padjust='bonf', parametric=True, within_first=False)

#pg.ttest(df[(df.exp_num==1) & (df.modality=='LE')].slope, df[(df.exp_num==1) & (df.modality=='ES')].slope, paired=True, confidence=.975)

#get CI 
for exp_num in (1, 2, 3, 4):
    for m1, m2 in (('ES', 'SP'), ):
        pg.ttest(df[(df.exp_num==exp_num) & (df.modality==m1)].slope, df[(df.exp_num==exp_num) & (df.modality==m2)].slope, paired=True, confidence=.975)



*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 3816
N cols= 4
   subject  exp_num   slope modality
0        1        1 0.42728       LE
1        2        1 0.90420       LE
2        3        1 0.43349       LE
3        4        1 0.65978       LE
4        5        1 0.84207       LE


/home/getz/.local/lib/python3.8/site-packages/pingouin/pairwise.py:27: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


,Contrast,exp_num,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,exp_num,-,1,2,False,True,0.33486,136.48096,two-sided,0.73824,1.00000,bonf,0.187,0.05468
1,exp_num,-,1,3,False,True,-3.14426,154.35771,two-sided,0.00200,0.01198,bonf,14.983,-0.50013
2,exp_num,-,1,4,False,True,-4.48892,161.60372,two-sided,0.00001,0.00008,bonf,1281.857,-0.69790
3,exp_num,-,2,3,False,True,-3.96087,149.68129,two-sided,0.00012,0.00069,bonf,187.433,-0.62656
4,exp_num,-,2,4,False,True,-5.39870,150.98446,two-sided,0.00000,0.00000,bonf,5.184e+04,-0.82489
5,exp_num,-,3,4,False,True,-1.57135,167.89774,two-sided,0.11798,0.70790,bonf,0.518,-0.23945
6,modality,-,ES,LE,True,True,-22.10008,317.00000,two-sided,0.00000,0.00000,bonf,3.819e+62,-1.64640
7,modality,-,ES,SP,True,True,-7.49413,317.00000,two-sided,0.00000,0.00000,bonf,8.571e+09,-0.33880
8,modality,-,LE,SP,True,True,15.24041,317.00000,two-sided,0.00000,0.00000,bonf,2.341e+36,1.09949
9,exp_num * modality,1,ES,LE,True,True,-6.53285,75.00000,two-sided,0.00000,0.00000,bonf,1.679e+06,-1.03975


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,-2.62616,75,two-sided,0.01046,"[-0.1788105925517567, -0.01233200432462586]",0.24814,3.068,0.56968


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,-3.42723,70,two-sided,0.00102,"[-0.1720800849137535, -0.034205902416627326]",0.37201,24.712,0.87104


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,-4.38873,82,two-sided,0.00003,"[-0.24921907937380333, -0.07864343324830048]",0.43474,537.982,0.97464


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,-4.45623,87,two-sided,0.00002,"[-0.2481399705483134, -0.08012331658699347]",0.35209,704.372,0.90426


# Fig 3

* load data
 

In [12]:
filename = 'Fig3D'
df = read_csv(filename)




*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 1624
N cols= 4
   subject  exp_num   slope modality
0        1  5.00000 0.06667       ES
1        2  5.00000 1.33418       ES
2        3  5.00000 0.05210       ES
3        4  5.00000 0.53807       ES
4        5  5.00000 0.92935       ES


* Fig3D, ES-EE slope ttest comparisons

In [13]:
pg.pairwise_tests(data=df, dv='slope', subject='subject', between='exp_num', within='modality', padjust='bonf', within_first=False, effsize='cohen')
for exp_num in (5, 6.1, 6.2):
    for m1, m2 in (('EE', 'ES'), ):
        pg.ttest(df[(df.exp_num==exp_num) & (df.modality==m1)].slope, df[(df.exp_num==exp_num) & (df.modality==m2)].slope, paired=True, confidence=.975)

,Contrast,exp_num,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,exp_num,-,5.00000,6.10000,False,True,-0.01296,134.93769,two-sided,0.98968,1.00000,bonf,0.183,-0.00221
1,exp_num,-,5.00000,6.20000,False,True,-1.38826,134.69286,two-sided,0.16735,0.50204,bonf,0.44,-0.23632
2,exp_num,-,6.10000,6.20000,False,True,-1.38645,130.00000,two-sided,0.16798,0.50395,bonf,0.446,-0.24135
3,modality,-,EE,ES,True,True,7.68370,202.00000,two-sided,0.00000,NaN,NaN,1.007e+10,0.50407
4,exp_num * modality,5.00000,EE,ES,True,True,4.48225,70.00000,two-sided,0.00003,0.00008,bonf,667.492,0.53396
5,exp_num * modality,6.10000,EE,ES,True,True,4.05588,65.00000,two-sided,0.00014,0.00041,bonf,159.138,0.40360
6,exp_num * modality,6.20000,EE,ES,True,True,4.84340,65.00000,two-sided,0.00001,0.00002,bonf,2136.461,0.57446


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,4.48225,70,two-sided,0.00003,"[0.10568190825675934, 0.3265957590813237]",0.53396,667.492,0.99337


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,4.05588,65,two-sided,0.00014,"[0.06368599820409925, 0.22961227802618078]",0.40360,159.138,0.89797


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,4.84340,65,two-sided,0.00001,"[0.10697115001152738, 0.29956201328996246]",0.57446,2136.461,0.99582



* Fig 3D, ES-EE slope regression across Exp 6.1-6.2

In [14]:
# test performance within exp and across cond with regression
print_section('regress slope across experiments')

df['exp_num'] = df['exp_num'].astype(str)
df = df[df.exp_num!='5.0']
df
# iterate through each modality and set it as "reference modality" to get "absolute slopes" for each
for modality in ('ES', 'EE'):
    print_subsection('Modality used as reference= ' + modality)
    smf.mixedlm(f'slope ~ C(exp_num, Treatment("6.1")) * C(modality, Treatment("{modality}"))', groups='subject', data=df).fit().summary()



*---------------------------------------------------------------------------------------------------------------------------------------*
* regress slope across experiments
*---------------------------------------------------------------------------------------------------------------------------------------*


,subject,exp_num,slope,modality
142,72,6.1,0.67574,ES
143,73,6.1,0.15000,ES
144,74,6.1,0.15023,ES
145,75,6.1,0.45003,ES
146,76,6.1,0.03425,ES
...,...,...,...,...
401,199,6.2,0.27926,EE
402,200,6.2,0.97121,EE
403,201,6.2,0.78354,EE
404,202,6.2,-0.21530,EE




***** Modality used as reference= ES *****
*---------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                     Mixed Linear Model Regression Results
===============================================================================================================
Model:                               MixedLM                    Dependent Variable:                    slope   
No. Observations:                    264                        Method:                                REML    
No. Groups:                          132                        Scale:                                 0.0506  
Min. group size:                     2                          Log-Likelihood:                        -80.8937
Max. group size:                     2                          Converged:                             Yes     
Mean group size:                     2.0                                                                       
---------------------------------------------------------------------------------------------------------------
                                                                       Coef. Std.Err.   z   P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------------------------
Intercept                                                              0.316    0.044 7.165 0.000  0.230  0.403
C(exp_num, Treatment("6.1"))[T.6.2]                                    0.049    0.062 0.789 0.430 -0.073  0.172
C(modality, Treatment("ES"))[T.EE]                                     0.147    0.039 3.744 0.000  0.070  0.223
C(exp_num, Treatment("6.1"))[T.6.2]:C(modality, Treatment("ES"))[T.EE] 0.057    0.055 1.022 0.307 -0.052  0.165
subject Var                                                            0.078    0.081                          
===============================================================================================================

"""



***** Modality used as reference= EE *****
*---------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                      Mixed Linear Model Regression Results
=================================================================================================================
Model:                                 MixedLM                    Dependent Variable:                    slope   
No. Observations:                      264                        Method:                                REML    
No. Groups:                            132                        Scale:                                 0.0506  
Min. group size:                       2                          Log-Likelihood:                        -80.8937
Max. group size:                       2                          Converged:                             Yes     
Mean group size:                       2.0                                                                       
-----------------------------------------------------------------------------------------------------------------
                                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                                               0.463    0.044 10.487 0.000  0.376  0.549
C(exp_num, Treatment("6.1"))[T.6.2]                                     0.106    0.062  1.696 0.090 -0.016  0.228
C(modality, Treatment("EE"))[T.ES]                                     -0.147    0.039 -3.744 0.000 -0.223 -0.070
C(exp_num, Treatment("6.1"))[T.6.2]:C(modality, Treatment("EE"))[T.ES] -0.057    0.055 -1.022 0.307 -0.165  0.052
subject Var                                                             0.078    0.081                           
=================================================================================================================

"""

# Fig 4

* Fig 4C and 4C ttest (ES - EE)

In [15]:
filename = 'Fig4CD'
df = read_csv(filename)
df
pg.pairwise_tests(data=df, dv='score', subject='subject', between='exp_num', within='modality', padjust='bonf', within_first=False, effsize='cohen')
pg.pairwise_tests(data=df, dv='score', subject='subject', between='exp_num', within='modality', padjust='bonf', within_first=True, effsize='cohen')

opp_exp = {6: 7, 7: 6}
for exp_num in (7, 6):
    for m1, m2 in (('EE', 'ED'), ('EE', 'EE'), ('ED', 'ED') ):
        pg.ttest(df[(df.exp_num==exp_num) & (df.modality==m1)].score, df[(df.exp_num==exp_num) & (df.modality==m2)].score, paired=True, confidence=.975)
        pg.ttest(df[(df.exp_num==opp_exp[exp_num]) & (df.modality==m1)].score, df[(df.exp_num==exp_num) & (df.modality==m2)].score, paired=True, confidence=.975)



*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 1096
N cols= 4
   subject  exp_num   score modality
0        1        6 0.95455       ED
1        1        6 0.89286       EE
2        2        6 0.85227       ED
3        2        6 0.68750       EE
4        3        6 0.85795       ED


,subject,exp_num,score,modality
0,1,6,0.95455,ED
1,1,6,0.89286,EE
2,2,6,0.85227,ED
3,2,6,0.68750,EE
4,3,6,0.85795,ED
...,...,...,...,...
269,135,7,0.53571,EE
270,136,7,0.51562,ED
271,136,7,0.49107,EE
272,137,7,0.85938,ED


,Contrast,exp_num,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,exp_num,-,6,7,False,True,5.69741,126.94145,two-sided,0.00000,NaN,NaN,1.589e+05,0.96271
1,modality,-,ED,EE,True,True,2.91943,136.00000,two-sided,0.00411,NaN,NaN,5.471,0.23661
2,exp_num * modality,6,ED,EE,True,True,11.55280,65.00000,two-sided,0.00000,0.00000,bonf,2.795e+14,1.31796
3,exp_num * modality,7,ED,EE,True,True,-6.36494,70.00000,two-sided,0.00000,0.00000,bonf,6.959e+05,-0.46424


,Contrast,modality,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,modality,-,ED,EE,True,True,2.91943,136.00000,two-sided,0.00411,NaN,NaN,5.471,0.23661
1,exp_num,-,6,7,False,True,5.69741,126.94145,two-sided,0.00000,NaN,NaN,1.589e+05,0.96271
2,modality * exp_num,ED,6,7,False,True,11.01610,94.97747,two-sided,0.00000,0.00000,bonf,2.117e+17,1.83542
3,modality * exp_num,EE,6,7,False,True,0.38425,131.77471,two-sided,0.70141,1.00000,bonf,0.196,0.06590


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,6.36494,70,two-sided,0.00000,"[0.039554451699244965, 0.08403080987017154]",0.46424,6.959e+05,0.97112


...

,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,-0.38425,131.77471,two-sided,0.70141,"[-0.057496581349998084, 0.04083220112928005]",0.06590,0.196,0.06694


/home/getz/.local/lib/python3.8/site-packages/pingouin/parametric.py:243: UserWarning: x and y are equals. Cannot compute T or p-value.
  warnings.warn("x and y are equals. Cannot compute T or p-value.")


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,NaN,65,two-sided,NaN,"[nan, nan]",0.00000,nan,0.05000


/home/getz/.local/lib/python3.8/site-packages/pingouin/parametric.py:227: UserWarning: x and y have unequal sizes. Switching to paired == False. Check your data.
  warnings.warn("x and y have unequal sizes. Switching to paired == False. Check your data.")


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,-11.01610,94.97747,two-sided,0.00000,"[-0.24744192101474338, -0.16265718948849625]",1.83542,2.117e+17,1.00000


* Fig 4E ttests (ES - EE)

In [16]:
filename = 'Fig4E'
df = read_csv(filename)

pg.pairwise_tests(data=df, dv='slope', subject='subject', within='modality', padjust='bonf', within_first=True, effsize='cohen')
for exp_num in (5, 6.1, 6.2):
    for m1, m2 in (('EE', 'ES'), ):
        pg.ttest(df[(df.exp_num==exp_num) & (df.modality==m1)].slope, df[(df.exp_num==exp_num) & (df.modality==m2)].slope, paired=True, confidence=.975)



*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 568
N cols= 4
   subject  exp_num    slope modality
0        1        7  0.34057       ES
1        2        7  0.00000       ES
2        3        7 -0.03665       ES
3        4        7  0.56014       ES
4        5        7  0.58632       ES


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,modality,EE,ES,True,True,5.80159,70.00000,two-sided,0.00000,7.863e+04,0.61063


AssertionError: y cannot be an empty list or array.

# Fig 5


* Fig5BC: Compare RT in Exp. 5:6  ES_s (ES pairs when S is chosen) with ES_e (when E is chosen) and EE

In [ ]:
filename = 'Fig5BC'
df = read_csv(filename)
df
pg.pairwise_tests(data=df, dv='RT', subject='subject', between='exp_num', within='modality', padjust='bonf', within_first=True, effsize='cohen')
for m1, m2 in (('EE', 'e'), ('EE', 'd'), ('e', 'd')):
    pg.ttest(df[(df.modality==m1)].RT, df[df.modality==m2].RT, paired=True, confidence=.975)



*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 1644
N cols= 4
   subject  exp_num          RT modality
0        1        5 2,130.00000        e
1        1        5 1,733.50000        d
2        1        5 1,923.50000       EE
3        2        5   957.00000        e
4        2        5 1,158.00000        d


,subject,exp_num,RT,modality
0,1,5,"2,130.00000",e
1,1,5,"1,733.50000",d
2,1,5,"1,923.50000",EE
3,2,5,957.00000,e
4,2,5,"1,158.00000",d
...,...,...,...,...
406,136,6,976.00000,d
407,136,6,793.50000,EE
408,137,6,"1,200.00000",e
409,137,6,"1,330.00000",d


,Contrast,modality,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,modality,-,EE,d,True,True,5.01785,136.00000,two-sided,0.00000,0.00000,bonf,7964.115,0.28126
1,modality,-,EE,e,True,True,3.29980,136.00000,two-sided,0.00124,0.00371,bonf,16.162,0.17523
2,modality,-,d,e,True,True,-3.23949,136.00000,two-sided,0.00151,0.00452,bonf,13.508,-0.12266
3,exp_num,-,5,6,False,True,1.44589,131.70634,two-sided,0.15058,NaN,NaN,0.474,0.24800
4,modality * exp_num,EE,5,6,False,True,1.39670,134.40921,two-sided,0.16481,0.49442,bonf,0.445,0.23875
5,modality * exp_num,d,5,6,False,True,1.34023,133.22731,two-sided,0.18245,0.54736,bonf,0.415,0.22951
6,modality * exp_num,e,5,6,False,True,1.37892,132.70087,two-sided,0.17024,0.51072,bonf,0.435,0.23628


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,3.29980,136,two-sided,0.00124,"[33.349669389290725, 179.6503306107093]",0.17523,16.162,0.53052


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,5.01785,136,two-sided,0.00000,"[92.68315573617076, 245.38253769448616]",0.28126,7964.115,0.90469


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,3.23949,136,two-sided,0.00151,"[18.782177621098974, 106.28351580955797]",0.12266,13.508,0.29687


* Fig 5D: compare RT of choices explained by exp. neglect vs  LE estimates

In [ ]:
filename = 'Fig5D'
df = read_csv(filename)
pg.ttest(df[(df.modality=='H')].RT, df[df.modality=='LE'].RT, paired=True, confidence=.975)
pg.pairwise_tests(dv='RT', within='modality', between='exp_num', subject='subject', data=df, padjust='bonf', within_first=True, effsize='cohen')



*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 3640
N cols= 4
   subject  exp_num          RT modality
0        1        1 4,207.00000        H
1        1        1 5,292.50000       LE
2        2        1   890.00000        H
3        2        1 1,380.00000       LE
4        3        1 2,449.00000        H


,T,dof,alternative,p-val,CI98%,cohen-d,BF10,power
T-test,-2.20288,385,two-sided,0.02820,"[-669.8522702921792, 7.119109670417621]",0.15550,0.626,0.86160


,Contrast,modality,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,modality,-,H,LE,True,True,-2.20288,385.00000,two-sided,0.02820,NaN,NaN,0.626,-0.15550
1,exp_num,-,1,2,False,True,1.26939,72.92436,two-sided,0.20834,1.00000,bonf,0.4,0.21406
2,exp_num,-,1,3,False,True,1.30799,68.41386,two-sided,0.19526,1.00000,bonf,0.398,0.23232
3,exp_num,-,1,4,False,True,0.82876,75.77422,two-sided,0.40984,1.00000,bonf,0.258,0.14269
4,exp_num,-,1,5,False,True,1.18100,70.44474,two-sided,0.24157,1.00000,bonf,0.35,0.20625
5,exp_num,-,1,6,False,True,1.39398,77.29040,two-sided,0.16732,1.00000,bonf,0.454,0.24079
6,exp_num,-,2,3,False,True,0.05788,98.07898,two-sided,0.95396,1.00000,bonf,0.19,0.01070
7,exp_num,-,2,4,False,True,-1.16776,114.83152,two-sided,0.24532,1.00000,bonf,0.362,-0.21306
8,exp_num,-,2,5,False,True,-0.31818,112.10212,two-sided,0.75094,1.00000,bonf,0.202,-0.05814
9,exp_num,-,2,6,False,True,0.38141,114.17545,two-sided,0.70361,1.00000,bonf,0.209,0.06893


# Review, fig1, test accuracy bad/good

In [ ]:
filename = 'corr_tables'
df = read_csv(filename)
pg.pairwise_ttests(dv='slope', subject='subject', between='exp_num', within='modality', 
            data=df, padjust='bonf', within_first=False, parametric=True)

# pg.print_table(res, floatfmt='.6f')






*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 592
N cols= 4
   subject  exp_num    slope modality
0        1        1  0.22351       ES
1        2        1  0.44697       ES
2        3        1 -0.06116       ES
3        4        1  0.52977       ES
4        5        1  0.23695       ES


c:\tools\Anaconda3\lib\site-packages\pingouin\pairwise.py:27: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


ValueError: Columns must have at least two unique values.

# Fig 2, test slope bad good

In [ ]:
filename = 'review_perf_fig2'
df = read_csv(filename)

pg.pairwise_ttests(dv='value', between=['exp_num', 'split'], subject='id',
            data=df, padjust='bonf', parametric=True)





In [ ]:
filename = 'review_accuracy'
df = read_csv(filename)

res = pg.pairwise_ttests(dv='value', between=['modality', 'split'], subject='id',
            data=df, padjust='bonf', parametric=True)

pg.print_table(res, floatfmt='.6f')


